In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import os,sys
from subprocess import Popen, PIPE

class AudioAugmentation(object):
    def __init__(self):
        self.sample_rate=16000
        

    def read_audio_file(self, file_path,noise_path):
        #input_length = 16000
        data = librosa.core.load(file_path)[0]
        noise_data = librosa.core.load(file_path)[0]
        if len(data) > input_length:
            data = data[:input_length]
        else:
            data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        return data
    
    def read_audio_files(self, file_path,noise_path):
        data = librosa.core.load(file_path)[0]
        noise_data = librosa.core.load(noise_path)[0]
        print(len(data),len(noise_data))
        if len(data) > len(noise_data):
            print("Exit None")
            #data = data[:input_length]
            return None
#         else:
#             data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        return (data,noise_data[:len(data)])

    def read_audio(self, file_path):
        #input_length = 16000
        fs, data = wavfile.read(file_path)
        self.sample_rate=fs
        return data
    
    def add_envrinment_noise(self,wav_paths):
        #noise = np.random.randn(len(data))
        main_data=self.read_audio(wav_paths[0])
        noise_data=self.read_audio(wav_paths[1])
        if len(main_data)>len(main_data):
            return None
        data_with_noise = main_data + noise_data[:len(main_data)]
        
        return data_with_noise

    def shift(self, data):
            return np.roll(data, 1600)

    def stretch(self, data, rate=1):
            input_length = 16000
            data = librosa.effects.time_stretch(data, rate)
            if len(data) > input_length:
                data = data[:input_length]
            else:
                data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
            return data
        
    def plot_time_series(self, data):
            fig = plt.figure(figsize=(14, 8))
            plt.title('Raw wave ')
            plt.ylabel('Amplitude')
            plt.plot(np.linspace(0, 1, len(data)), data)
            plt.show()
            
    def audio_convert_to_wav(self,input_path,out_path,sample_rate=8000):
        if  os.path.exists(input_path):

            process = Popen(
                ['ffmpeg', '-i', input_path,'-acodec','pcm_s16le','-ac','1', '-ar',sample_rate, out_path],
                stdout=PIPE, stderr=PIPE)
            stdout, stderr = process.communicate()
            print("Done AUDIO CONVERT 16 16k mono.......")
            
#     def write_audio_file(self, file, data, sample_rate=8000):
#             librosa.output.write_wav(file, data, sample_rate)
            
    def write_audio(self,output_wav,signal,sampleRate=8000):
        wavfile.write(output_wav, sampleRate, signal)
        
    def task(self,wav_paths):
        injected_noise=self.add_envrinment_noise(wav_paths)
        if injected_noise is not None:
            self.write_audio(wav_paths[2],injected_noise,sampleRate=8000)
            


  


# List of Audio File

In [ ]:
import glob
wav_files=glob.glob("/home/ubuntu/projects/deepspeech.pytorch_plus_plus/data/LibriSpeech_dataset/test_clean/wav/*.wav")
#print(wav_files[:10])
noise_files=glob.glob("/home/ubuntu/projects/noise_data/wav/*.wav")
noise_files

In [ ]:
import random
random.choice(noise_files)

In [ ]:
from concurrent import futures
import os
import random
wav_paths=[]
save_aug_wav="/home/ubuntu/projects/datasets/injected_wav"
for i in range(len(wav_files[:10])):
    t=[]
    t.append(wav_files[i])
    t.append(random.choice(noise_files))
    t.append(os.path.join(save_aug_wav+"/aug__"+wav_files[i].split('/')[-1]))
    wav_paths.append(t)
print(wav_paths[0])


In [ ]:
aa = AudioAugmentation()
aa.task(wav_paths[0])

In [ ]:
aa = AudioAugmentation()
with futures.ProcessPoolExecutor(max_workers=2) as ex:
    results = ex.map(aa.task,wav_paths )
#     for n, pid in results:
#         print('ran task {} in process {}'.format(n, pid))

In [ ]:
import glob
wav_files=glob.glob("/home/ubuntu/projects/deepspeech.pytorch_plus_plus/data/LibriSpeech_dataset/test_clean/wav/*.wav")
print(wav_files[0])
noise_files=glob.glob("/home/ubuntu/projects/noise_data/wav/*.wav")
noise_files

# Convert mp3 to wav 

In [ ]:
import os,sys
from subprocess import Popen, PIPE
def audio_convert(input_path,out_path):
    if  os.path.exists(input_path):
        
        process = Popen(
            ['ffmpeg', '-i', input_path,'-acodec','pcm_s16le','-ac','1', '-ar','8000', out_path],
            stdout=PIPE, stderr=PIPE)
        stdout, stderr = process.communicate()
        print("Done AUDIO CONVERT 16 16k mono.......")

In [ ]:
out_path=noise_files[5].split('.')[0]+"_8k.wav"
audio_convert(noise_files[5],out_path)
out_path

# Play audio

In [ ]:
import IPython
IPython.display.Audio(noise_files[5])

In [ ]:
#!sox --i '/home/ubuntu/projects/noise_data/wav/wav56tmi_8k.wav'
!sox --i "/home/ubuntu/brain_cornell_8k.wav"

In [ ]:
import IPython
IPython.display.Audio(wav_files[0])

# Processing

In [ ]:
aa = AudioAugmentation()

# Read cat sound
data = aa.read_audio_files(wav_files[0],out_path)
#aa.plot_time_series(data[1][:16000])

In [ ]:
aa.plot_time_series(data[1][:16000])

In [ ]:
from scipy.io import wavfile
fs, data_s = wavfile.read("/home/ubuntu/brain_cornell_8k.wav")
data_s2 = aa.read_audio("/home/ubuntu/brain_cornell_8k.wav")
noise_data_scipy = wavfile.read(out_path)
print(type(data_s),type(data_s2),len(data_s),len(data_s2),len(noise_data_scipy[1]))


In [ ]:
print(type(data_s),type(data_s2),len(data_s),len(data_s2),len(noise_data_scipy[1]))
add_noise_data=data_s+noise_data_scipy[1][:len(data_s)]

In [ ]:
aa.plot_time_series(add_noise_data[:32000])

In [ ]:
aa.plot_time_series(data_s[:32000])

In [ ]:
aa.plot_time_series(data_s2[:96000])

In [ ]:
import IPython
IPython.display.Audio("/home/ubuntu/projects/datasets/injected_wav/aug__1221-135767-0006.wav")

In [ ]:
type(data[1])
data[1][:1000]

In [ ]:
a=np.array([1,2,3,1,2,3,1,1,3])
b=np.array([4,5,3,5,6,5,6,4,6])
b

In [ ]:
a+b

In [ ]:
# Adding noise to sound
data_noise = aa.add_noise(data[0],data[1])
aa.write_audio_file('file_noise_librosa.wav', data_noise, sample_rate=8000)

In [3]:
import IPython
IPython.display.Audio('/home/ubuntu/projects/datasets/LibriSpeech/test-clean-wav/7127-75947-0000.wav')

In [6]:
import IPython
IPython.display.Audio('/home/ubuntu/projects/audio_labelling/notebook/saved_audio_with_noise_file.wav')

In [ ]:
data_s = aa.read_audio("/home/ubuntu/brain_cornell_8k.wav")


In [ ]:
aa.plot_time_series(data_s[:16000])

In [ ]:
data = aa.read_audio_files("/home/ubuntu/projects/audio_labelling/notebook/file_noise.wav",out_path)
aa.plot_time_series(data[0][:64000])

# Tensorflow audio processing

In [5]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import, division, print_function

import os

from functools import partial

import numpy as np
import pandas
import tensorflow as tf
import datetime

from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio

saved_audio_with_noise_file = tf.constant('saved_audio_with_noise_file')
samples = tf.read_file("/home/ubuntu/projects/datasets/LibriSpeech/test-clean-wav/7127-75947-0000.wav")
decoded = contrib_audio.decode_wav(samples, desired_channels=1)
samples1 = tf.read_file("/home/ubuntu/projects/datasets/noise_wav/00noice.wav")
noise_decoded = contrib_audio.decode_wav(samples1, desired_channels=1)





print(tf.size(decoded.sample_rate))
print(tf.size(noise_decoded.sample_rate))

if_true_cond = tf.cond(tf.size(decoded.audio) > tf.size(noise_decoded.audio), lambda : decoded.audio, lambda : tf.add(decoded.audio, tf.slice(noise_decoded.audio,[0,0],tf.unstack(tf.shape(decoded.audio)))))

print(tf.shape(tf.slice(noise_decoded.audio,[0,0],tf.unstack(tf.shape(decoded.audio)))))
print(tf.shape(decoded.audio))

decoded_audio = tf.cond(tf.equal(decoded.sample_rate, noise_decoded.sample_rate), lambda: if_true_cond, lambda: decoded.audio)
decoded_audio = tf.identity(decoded_audio, name="input_with_noise_audio")


encoded=tf.contrib.ffmpeg.encode_audio(
    decoded_audio,
    file_format='wav',
    samples_per_second=16000
)


write_file_op = tf.write_file(saved_audio_with_noise_file, encoded)


# aa = AudioAugmentation()
# sess = tf.Session()
# with sess.as_default():
#     print(type(decoded.sample_rate.eval()),type(noise_decoded.sample_rate.eval()))
#     print(np.dtype(decoded.audio.eval()[0][0]))
#     print(len(decoded.audio.eval()),len(noise_decoded.audio.eval()),len(decoded_audio.eval()))
#     print(decoded.audio.eval().ravel())#.reshape((len(noise_decoded.audio.eval().ravel()),1)))
#     write_file_op
#     aa.plot_time_series(decoded.audio.eval().ravel()[:64000])
#     aa.plot_time_series(decoded_audio.eval().ravel()[:64000])
    
    
with tf.Session() as sess:
    sess.run(write_file_op)

Tensor("Size_8:0", shape=(), dtype=int32)
Tensor("Size_9:0", shape=(), dtype=int32)
Tensor("Shape_7:0", shape=(2,), dtype=int32)
Tensor("Shape_8:0", shape=(2,), dtype=int32)


In [ ]:
input_filename = tf.placeholder(tf.string, shape=[])
output_filename = tf.placeholder(tf.string, shape=[])

input_signal = tf.contrib.ffmpeg.decode_audio(
    tf.read_file(input_filename), file_format="wav",
    samples_per_second=44100, channel_count=2)

# ...

output_signal = ...  # A 2-D tensor, [samples x channels]
encoded_audio_data = tf.contrib.ffmpeg.encode_audio(
    output_signal, file_format="wav", samples_per_second=44100)

write_file_op = tf.write_file(output_filename, encoded_audio_data)

with tf.Session() as sess:
  sess.run(write_file_op, {input_filename: "input.wav",
                           output_filename: "output.wav"})

In [4]:
import pandas as pd
from shutil import copyfile

In [12]:
pa="/home/ubuntu/projects/deepspeech.pytorch/data/TEDLIUM_dataset/TEDLIUM_release2/test/converted"
sav="/home/ubuntu/projects/sample_data"
import glob
wav_l=glob.glob(pa+"/wav/*.wav")

In [27]:
k=1
for i in wav_l[:100]:
#     print(i)
    copyfile(pa+"/txt/"+i.split("/")[-1].split('.')[0]+".txt",sav+"/txt/"+str(k)+".txt")
#     copyfile(i,sav+"/wav/"+str(k)+".wav")

    k+=1
    

In [28]:
!ls "/home/ubuntu/projects/sample_data/txt"

100.txt  19.txt  28.txt  37.txt  46.txt  55.txt  64.txt  73.txt  82.txt  91.txt
10.txt	 1.txt	 29.txt  38.txt  47.txt  56.txt  65.txt  74.txt  83.txt  92.txt
11.txt	 20.txt  2.txt	 39.txt  48.txt  57.txt  66.txt  75.txt  84.txt  93.txt
12.txt	 21.txt  30.txt  3.txt	 49.txt  58.txt  67.txt  76.txt  85.txt  94.txt
13.txt	 22.txt  31.txt  40.txt  4.txt	 59.txt  68.txt  77.txt  86.txt  95.txt
14.txt	 23.txt  32.txt  41.txt  50.txt  5.txt	 69.txt  78.txt  87.txt  96.txt
15.txt	 24.txt  33.txt  42.txt  51.txt  60.txt  6.txt	 79.txt  88.txt  97.txt
16.txt	 25.txt  34.txt  43.txt  52.txt  61.txt  70.txt  7.txt	 89.txt  98.txt
17.txt	 26.txt  35.txt  44.txt  53.txt  62.txt  71.txt  80.txt  8.txt	 99.txt
18.txt	 27.txt  36.txt  45.txt  54.txt  63.txt  72.txt  81.txt  90.txt  9.txt
